<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/INFORME_CU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Informe Provincial Cuenca
##Descarga y limpeza de datos

Comenzamos cargando los datos de la carpeta de Google Drive en la que estén guardados (será necesario cambiar esta ruta en función de la ruta del ordenador).

Es necesario aceptar la conexión con la cuenta de Google Drive.

Importante y esencial, antes de tirar el código es necesario que la carpeta de Drive a la que se va a llamar tenga los archivos en formato xlsx así nombrados: 'CU_SAAF.xlsx', 'CU_AT.xlsx', 'CU_EMPLEO.xlsx', 'CU_LOG.xlsx' para que no haya errores a la hora de ejecutar este código.

**¡OJO! Primeros archivo con extensión xlsx y los de las zonas a clasificar, con extensión csv, en teoría, los csv no deberían modificarse (a menos que las normativas cambien y con ello las zonas de impacto se modifiquen).**

**Y todos las columnas deben tener los nombres definidos en el documento "Resumen de datos por servicio FASPAS" y en mayúsculas antes de ser subidos al programa.**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: The domain policy has disabled Drive File Stream: https://support.google.com/a/answer/7496409

Se importan las librerías necesarias para la parte de carga de datos y su limpieza.

In [4]:
import os
import pandas as pd
import numpy as np

In [ ]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/CU'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path_csv = '/content/drive/My Drive/PFG_FASPAS/Zonas_Prioritarias'
docs_csv = [f for f in os.listdir(folder_path_csv) if f.endswith('.csv')]
print(docs_csv)

['CU_SAAF.xlsx', 'CU_AT.xlsx', 'CU_EMPLEO.xlsx', 'CU_LOG.xlsx']
['municipios_prioritarios_clm.csv']


Creación de un diccionario que almacene los dataframes (.xlsx) contenidos en la carpeta de Cuenca.

Se irán cargando todos los archivos en un dataframe que se añade al diccionario con el nombre de dicho archivo como clave.

In [ ]:
dic_dataframes = {}
dic_zonas = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

for doc in docs_csv:
  entire_path = os.path.join(folder_path_csv, doc)
  df = pd.read_csv(entire_path)
  # Uso el nombre del archivo como clave
  dic_zonas[doc] = df

A continuación se debe observar que el dataframe cuente con al menos las columnas: 'AÑOS ATENCIÓN', 'FECHA NACIMIENTO', 'LOCALIDAD', 'GÉNERO' y 'CP'

Ahora que sabemos cuáles son las claves, guardamos cada dataframe por separado para trabajar con todos ellos de manera individual, por el momento.

In [ ]:
# Imprimir todas las claves
for clave in dic_dataframes.keys():
    print(clave)

CU_SAAF.xlsx
CU_AT.xlsx
CU_EMPLEO.xlsx
CU_LOG.xlsx


In [5]:
CU_SAAF = dic_dataframes.get("CU_SAAF.xlsx")
CU_AT = dic_dataframes.get("CU_AT.xlsx")
CU_EMPLEO = dic_dataframes.get("CU_EMPLEO.xlsx")
CU_LOG = dic_dataframes.get("CU_LOG.xlsx")
# CU_SOCIOS = dic_dataframes.get("CU_SOCIOS.xlsx")

Vamos a limpiar cada uno de los dataframes en función de los datos que tenemos y vamos a normalizarlos para cuando los crucemos entre ellos.

Comenzamos con los datos de Servicio de Atención y Apoyo a Familias (SAAF).

Sacamos la información del dataset y observamos que de 77 entradas, el campo de 'GRADO DEPENDENCIA' tan solo tiene 1 registro, por lo que se procede a eliminarla. El 'GRADO DISCAPACIDAD' de momento lo dejamos ya que para el impacto territorial no lo vamos a usar.

In [6]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
CU_SAAF = pd.read_excel('/content/CU_SAAF.xlsx')
CU_AT = pd.read_excel('/content/CU_AT.xlsx')
CU_EMPLEO = pd.read_excel('/content/CU_EMPLEO.xlsx')
CU_LOG = pd.read_excel('/content/CU_LOG.xlsx')
# CU_SOCIOS = pd.read_excel('/content/CU_SOCIOS.xlsx')

In [13]:
# Lista de columnas necesarias para hacer el impacto territorial en todos los dataframes
needed_columns = ['FECHA NACIMIENTO', 'LOCALIDAD', 'GÉNERO', 'CP']

# Asegurarse de que todas las columnas necesarias existan en todos los dataframes, creándolas si es necesario
for col in needed_columns:
    if col not in CU_SAAF.columns:
        CU_SAAF[col] = np.nan  # Creamos la columna con valores NaN para que se ejecute el código
    if col not in CU_AT.columns:
        CU_AT[col] = np.nan
    if col not in CU_EMPLEO.columns:
        CU_EMPLEO[col] = np.nan
    if col not in CU_LOG.columns:
        CU_LOG[col] = np.nan
    # if col not in CU_SOCIOS.columns:
    #     CU_SOCIOS[col] = np.nan

CU_SAAF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   AÑO ATENCIÓN            77 non-null     int64  
 1   FECHA NACIMIENTO        77 non-null     object 
 2   GÉNERO                  77 non-null     object 
 3   LOCALIDAD               76 non-null     object 
 4   CP                      72 non-null     Int64  
 5   TIPO SORDERA            77 non-null     object 
 6   GRADO PÉRDIDA           59 non-null     object 
 7   MOMENTO APARICIÓN       61 non-null     object 
 8   TIPO PRÓTESIS           77 non-null     object 
 9   MODALIDAD COMUNICATIVA  77 non-null     object 
 10  GRADO DEPENDENCIA       1 non-null      object 
 11  GRADO DISCAPACIDAD      32 non-null     float64
dtypes: Int64(1), float64(1), int64(1), object(9)
memory usage: 7.4+ KB


Lo primero que hacemos es eliminar aquellas columnas cuyos datos NO se hayan pedido.

In [18]:
# Lista de columnas que me quedo independientemente de las que tenga el pdf
# OJOOOOOO: volver a mirar los archivos originales para normalizar todo (¿Lleva implante de tronco cerebral/osteointegrado/de oído medio?...
columns_to_keep = ['FECHA ALTA', 'FECHA NACIMIENTO', 'LOCALIDAD', 'GÉNERO', 'CP', 'TIPO SORDERA', 'MOMENTO APARICIÓN', '¿LLEVA AUDÍFONO?', 'AUD O PRÓTESIS', 'MODALIDAD COMUNICATIVA', 'GRADO DISCAPACIDAD', 'TIPO PRÓTESIS', 'GRADO PÉRDIDA']

# Solo las que estén contenidas en el df
CU_SAAF = CU_SAAF[CU_SAAF.columns.intersection(columns_to_keep)]

Por otra parte, la columna 'FECHA NACIMIENTO' nos quedamos solo con la fecha (OJOOOO ¿O SOLO EL AÑO????), eliminando la hora. Por defecto, Excel pone el CP como float, asi que lo pasamos a entero.

In [19]:
CU_SAAF['CP'] = CU_SAAF['CP'].astype('Int64')

CU_SAAF['FECHA NACIMIENTO'] = CU_SAAF['FECHA NACIMIENTO'].astype(str)

CU_SAAF['FECHA NACIMIENTO'] = CU_SAAF['FECHA NACIMIENTO'].str.split(' ').str[0]

<ipython-input-19-0a10314fec0d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CU_SAAF['CP'] = CU_SAAF['CP'].astype('Int64')
<ipython-input-19-0a10314fec0d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CU_SAAF['FECHA NACIMIENTO'] = CU_SAAF['FECHA NACIMIENTO'].astype(str)
<ipython-input-19-0a10314fec0d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

Si existe en el dataframe, como en este caso, aquellos que no tienen registro de 'TIPO PRÓTESIS' hay que examinar el motivo. Vamos a ver aquellos registros distintos que tienen todas las columnas. Vemos que en la columna 'TIPO PRÓTESIS' no hay ninguna salida que sea "NO TIENE" y preguntamos a FASPAS, tras su aprobación, procedemos a sustituir los valores nulos por 'NO TIENE'.

In [20]:
column_names = CU_SAAF.columns.tolist()

for column in column_names:
  unique_val = CU_SAAF[column].unique()
  print(unique_val)

['1993-02-05' '1975-07-10' '28/22/1995' '2008-09-16' '2008-09-25'
 '2007-10-26' '2008-08-08' '1995-08-02' '2005-10-13' '2010-12-29'
 '2006-11-17' '1999-10-14' '2010-12-14' '1961-05-20' '1990-09-11'
 '2003-08-26' '2008-12-22' '2010-03-04' '1982-05-24' '2011-10-29'
 '2012-12-12' '2012-12-14' '1950-08-26' '1959-06-18' '2014-01-02'
 '2012-09-17' '2013-09-04' '2007-09-19' '1976-05-12' '2012-05-24'
 '1961-08-14' '2009-01-01' '2015-11-11' '2016-05-18' '2014-12-28'
 '2006-03-12' '2016-08-11' '1986-08-13' '2016-10-10' '2017-02-07'
 '2016-01-15' '1951-10-13' '2006-04-22' '2017-10-13' '1935-04-12'
 '2018-06-20' '2018-03-02' '1945-08-10' '1949-08-28' '1941-06-22'
 '2008-04-07' '1955-06-12' '1993-03-05' '2019-12-19' '2020-03-31'
 '1997-10-02' '1992-01-19' '2019-07-15' '1974-12-07' '2021-04-07'
 '2020-06-05' '2019-07-29' '2018-10-20' '2020-07-01' '2020-04-23'
 '2020-04-15' '2020-03-03' '2020-07-23' '2020-11-04' '2020-10-15'
 '1905-06-25' '2016-09-04' '2018-07-31' '2008-08-22' '1969-07-12'
 '1946-05-

Se observa que la 'MODALIDAD COMUNICATIVA' no da ninguna información, pues todos los registros son "ORAL", sim embargo, la dejamos por si acaso en un futuro los registros cambiaran.

Y procedemos a poner todos los valores de los registros en mayúsculas, para uniformar los datos.

In [21]:
for column in CU_SAAF.columns:
    if CU_SAAF[column].dtype == 'object':
        CU_SAAF[column] = CU_SAAF[column].str.upper()

<ipython-input-21-5330a7e415d9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CU_SAAF[column] = CU_SAAF[column].str.upper()


Se disminuye la complejidad de los datos de manera que solo nos quedamos con la categoría de 'AUD O PRÓTESIS' que indica si lleva o no alguno de los dispotivos posibles (¿Lleva implante de tronco cerebral/osteointegrado/de oído medio?) con independencia de cuál de estos sea.

In [16]:
# HAY QUE CONTEMPLAR TOOOODAS LAS ENTRADAS!!!
if 'TIPO PRÓTESIS' in CU_SAAF.columns:
  CU_SAAF['TIPO PRÓTESIS'] = CU_SAAF['TIPO PRÓTESIS'].fillna('NO TIENE')
  CU_SAAF['AUD O PRÓTESIS'] = np.where(CU_SAAF['TIPO PRÓTESIS'] == 'NO TIENE', 'NO', 'SI')
  CU_SAAF.drop('TIPO PRÓTESIS', axis=1, inplace=True)

elif '¿LLEVA AUDÍFONO?' in CU_SAAF.columns:
  si = ['SI', 'SÍ', 'YES', 'S']
  for s in si:
    CU_SAAF['AUD O PRÓTESIS'] = np.where(CU_SAAF['¿LLEVA AUDÍFONO?'] == si, 'SI', 'NO')

elif 'AUD O PRÓTESIS' in CU_SAAF.columns:
  si = ['SI', 'SÍ', 'YES', 'S']
  no = ['NO', 'N']

  for s in si:
    CU_SAAF['AUD O PRÓTESIS'] = CU_SAAF['AUD O PRÓTESIS'].replace(s, 'SI')

  for n in no:
    CU_SAAF['AUD O PRÓTESIS'] = CU_SAAF['AUD O PRÓTESIS'].replace(n, 'NO')

Para contemplar que otros datos que se ingresen sean distintos, vamos a jugar con eliminar aquellas columnas que no sean esenciales para el estudio territorial, sino que nos den información más sociológica y sirvan para un estudio social; siempre y cuando el porcentaje de valores nulos sea mayor o igual que 1/3 del total de registros.

In [ ]:
# Añado 'AÑO ATENCIÓN' para SAAF
needed_columns = ['AÑO ATENCIÓN', 'FECHA NACIMIENTO', 'LOCALIDAD', 'GÉNERO', 'CP']

# Creación de la lista de columnas consideradas para posible eliminación
social_columns = [col for col in CU_SAAF.columns if col not in needed_columns]

# Porcentaje máximo de valores nulos permitido
max_percentage = 1/3

# Identifico columnas para eliminar
del_columns = []
for col in social_columns:
    if CU_SAAF[col].isnull().sum() / len(df) >= max_percentage:
        del_columns.append(col)

# Eliminar las columnas identificadas
CU_SAAF.drop(columns=del_columns, inplace=True)

Además, pondremos todos los registros de MUJER como M y HOMBRE como H. Esto lo realizaremos con todos los dataframes. Contemplamos que los registros que llegan pueden tener otros nombres en este campo.

In [27]:
if 'GÉNERO' in CU_SAAF.columns:
  # OJO: contemplar tb las diferentes formas de escribir TODAS LAS COLUMNAS 'MOMENTO APARICIÓN', HAY QUE NORMALIZAR TODO
  mujer = ['MUJER', 'HEMBRA', 'FEMENINO', 'FEM', 'F']
  hombre = ['GOMBRE', 'MACHO', 'MASCULINO', 'MASC', 'MAS']

  for m in mujer:
    CU_SAAF['GÉNERO'] = CU_SAAF['GÉNERO'].replace(m, 'M')

  for h in hombre:
    CU_SAAF['GÉNERO'] = CU_SAAF['GÉNERO'].replace(h, 'H')

<ipython-input-27-a65fe868edea>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CU_SAAF['GÉNERO'] = CU_SAAF['GÉNERO'].replace(m, 'M')
<ipython-input-27-a65fe868edea>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CU_SAAF['GÉNERO'] = CU_SAAF['GÉNERO'].replace(h, 'H')


Para el resto de columnas que todavía existan procedo a normalizar los registros teniendo en cuenta todos los posibles valores a tomar.

In [22]:
if 'TIPO SORDERA' in CU_SAAF.columns:
  # Lista con las posibles salidas permitidas
  strings_to_keep = ['NEUROSENSORIAL', 'CONDUCTIVA', 'MIXTA', 'TEL']

  # Sustituyo los registros distintos por la palabra 'NO REGISTRADO' (VER SI SE CAMBIA ESTO)
  CU_SAAF.loc[~CU_SAAF['TIPO SORDERA'].isin(strings_to_keep), 'TIPO SORDERA'] = 'NO REGISTRADO'

if 'MOMENTO APARICIÓN' in CU_SAAF.columns:
  strings_to_keep = ['PERILOCUTIVA', 'POSTLOCUTIVA', 'PRELOCUTIVA']

  # Sustituyo los registros distintos por la palabra 'NO REGISTRADO' (VER SI SE CAMBIA ESTO)
  CU_SAAF.loc[~CU_SAAF['MOMENTO APARICIÓN'].isin(strings_to_keep), 'MOMENTO APARICIÓN'] = 'NO REGISTRADO'

if 'MODALIDAD COMUNICATIVA' in CU_SAAF.columns:
  strings_to_keep = ['SIGNO', 'LSE','SIGNOS', 'ORAL', 'BIMODAL', 'L.S.E.', 'L.S.E', 'SIGNOS NATURALES', 'ORAL,SIGNOS NATURALES', 'ORAL, SIGNOS NATURALES']

  # Sustituyo los registros distintos por la palabra 'NO REGISTRADO' (VER SI SE CAMBIA ESTO)
  CU_SAAF.loc[~CU_SAAF['MOMENTO APARICIÓN'].isin(strings_to_keep), 'MOMENTO APARICIÓN'] = 'NO REGISTRADO'

  # Normalizo todas las formas de llamar a las distintas modalidades comunicativas
  ORAL = ['ORAL', 'ORAL,SIGNOS NATURALES', 'ORAL, SIGNOS NATURALES']
  LSE = ['LSE', 'L.S.E.','L.S.E']
  SIGNOS = ['SIGNO', 'SIGNOS', 'SIGNOS NATURALES']

  for o in ORAL:
    CU_SAAF['MODALIDAD COMUNICATIVA'] = CU_SAAF['MODALIDAD COMUNICATIVA'].replace(o, 'ORAL')

  for l in LSE:
    CU_SAAF['MODALIDAD COMUNICATIVA'] = CU_SAAF['MODALIDAD COMUNICATIVA'].replace(l, 'LSE')

  for s in SIGNOS:
    CU_SAAF['MODALIDAD COMUNICATIVA'] = CU_SAAF['MODALIDAD COMUNICATIVA'].replace(s, 'SIGNOS')

if 'GRADO PÉRDIDA' in CU_SAAF.columns:
  CU_SAAF['GRADO PÉRDIDA'] = CU_SAAF['GRADO PÉRDIDA'].astype(str)

  # Selecciono solo las 3 primeras letras (quitando la parte de DB si la tuviera)
  CU_SAAF['GRADO PÉRDIDA'] = CU_SAAF['GRADO PÉRDIDA'].str.slice(0, 3)

  strings_to_keep = ['DAP', 'DAM', 'DAS', 'DAL']

  CU_SAAF.loc[~CU_SAAF['GRADO PÉRDIDA'].isin(strings_to_keep), 'GRADO PÉRDIDA'] = 'NO REGISTRADO'

<ipython-input-22-c05c5f012cf1>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CU_SAAF['MODALIDAD COMUNICATIVA'] = CU_SAAF['MODALIDAD COMUNICATIVA'].replace(o, 'ORAL')
<ipython-input-22-c05c5f012cf1>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CU_SAAF['MODALIDAD COMUNICATIVA'] = CU_SAAF['MODALIDAD COMUNICATIVA'].replace(l, 'LSE')
<ipython-input-22-c05c5f012cf1>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [28]:
column_names = CU_SAAF.columns.tolist()

for column in column_names:
  unique_val = CU_SAAF[column].unique()
  print(unique_val)

['1993-02-05' '1975-07-10' '28/22/1995' '2008-09-16' '2008-09-25'
 '2007-10-26' '2008-08-08' '1995-08-02' '2005-10-13' '2010-12-29'
 '2006-11-17' '1999-10-14' '2010-12-14' '1961-05-20' '1990-09-11'
 '2003-08-26' '2008-12-22' '2010-03-04' '1982-05-24' '2011-10-29'
 '2012-12-12' '2012-12-14' '1950-08-26' '1959-06-18' '2014-01-02'
 '2012-09-17' '2013-09-04' '2007-09-19' '1976-05-12' '2012-05-24'
 '1961-08-14' '2009-01-01' '2015-11-11' '2016-05-18' '2014-12-28'
 '2006-03-12' '2016-08-11' '1986-08-13' '2016-10-10' '2017-02-07'
 '2016-01-15' '1951-10-13' '2006-04-22' '2017-10-13' '1935-04-12'
 '2018-06-20' '2018-03-02' '1945-08-10' '1949-08-28' '1941-06-22'
 '2008-04-07' '1955-06-12' '1993-03-05' '2019-12-19' '2020-03-31'
 '1997-10-02' '1992-01-19' '2019-07-15' '1974-12-07' '2021-04-07'
 '2020-06-05' '2019-07-29' '2018-10-20' '2020-07-01' '2020-04-23'
 '2020-04-15' '2020-03-03' '2020-07-23' '2020-11-04' '2020-10-15'
 '1905-06-25' '2016-09-04' '2018-07-31' '2008-08-22' '1969-07-12'
 '1946-05-

Aquí podemos ver cuáles son las columnas que se tendrán en cuenta para este caso y sus datos.

In [ ]:
CU_SAAF.info() # ESTO SE PUEDE BORRAR
column_names = CU_SAAF.columns.tolist()
print('\n')

for column in column_names:
  unique_val = CU_SAAF[column].unique()
  print(unique_val)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   AÑO ATENCIÓN            77 non-null     int64 
 1   FECHA NACIMIENTO        77 non-null     object
 2   GÉNERO                  77 non-null     object
 3   LOCALIDAD               77 non-null     object
 4   CP                      74 non-null     Int64 
 5   TIPO SORDERA            77 non-null     object
 6   MOMENTO APARICIÓN       61 non-null     object
 7   TIPO PRÓTESIS           77 non-null     object
 8   MODALIDAD COMUNICATIVA  77 non-null     object
dtypes: Int64(1), int64(1), object(7)
memory usage: 5.6+ KB


[2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]
['1993-02-05' '1975-07-10' '28/22/1995' '2008-09-16' '2008-09-25'
 '2007-10-26' '2008-08-08' '1995-08-02' '2005-10-13' '2010-12-29'
 '2006-11-17' '1999-10-14' '2010-12-14' '1961-05-20' '1990-0

**OJOOOOOOOO HAY QUE VER CÓMO PROCEDER PARA AQUELLOS REGISTROS CON DATOS NULOS EN LOS CAMPOS DE CP Y LOCALIDAD**

¿¿SI HAY UNO DE LOS DOS DEJAMOS EL REGISTRO, PERO SI NO HAY NADA EN AMBOS CAMPOS LOS ELIMINAMOS??

Se crea un filtro booleano que coge aquellos registros con ambos campos nulos y se lo aplicamos al dataframe. Ya que aquellos registros sin geolocalización no nos sirven para el objetivo principal que es saber el impacto que se tiene en las diferentes zonas para desarrollar un plan de ampliación territorial estratégico basado en el estudio estadístico.

In [ ]:
CU_SAAF = CU_SAAF.dropna(subset=['CP', 'LOCALIDAD'], how='all')

CP_loc_filter = CU_SAAF['CP'].isnull() & CU_SAAF['LOCALIDAD'].isnull()

CU_SAAF = CU_SAAF[~CP_loc_filter]

In [ ]:
CU_SAAF

,AÑO ATENCIÓN,FECHA NACIMIENTO,GÉNERO,LOCALIDAD,CP,TIPO SORDERA,MOMENTO APARICIÓN,TIPO PRÓTESIS,MODALIDAD COMUNICATIVA
0,2011,1993-02-05,M,NOHALES,16191,NEUROSENSORIAL,PRELOCUTIVA,2 IMPLANTES COCLEARES,ORAL
1,2011,1975-07-10,M,VALVERDE DEL JÚCAR,16100,NEUROSENSORIAL,POSTLOCUTIVA,2 IMPLANTES COCLEARES,ORAL
2,2011,28/22/1995,M,CARBONERAS,16350,NEUROSENSORIAL,PRELOCUTIVA,2 IMPLANTES COCLEARES,ORAL
3,2011,2008-09-16,M,CUENCA,16004,MIXTA,PRELOCUTIVA,2 IMPLANTES COCLEARES,ORAL
4,2011,2008-09-25,M,CUENCA,16003,NEUROSENSORIAL,PRELOCUTIVA,2 IMPLANTES COCLEARES,ORAL
...,...,...,...,...,...,...,...,...,...
72,2018,2018-07-31,HOMBRE,VILLALBA DE LA SIERRA,16140,CONDUCTIVA,PRELOCUTIVA,NO TIENE,ORAL
73,2018,2008-08-22,HOMBRE,CUENCA,16004,MIXTA,PRELOCUTIVA,NO TIENE,ORAL
74,2019,1969-07-12,M,REILLO,16390,NEUROSENSORIAL,POSTLOCUTIVA,AUDIFONO + IC,ORAL
75,2024,1946-05-13,M,VILLAR DE OLALLA,16195,NEUROSENSORIAL,POSTLOCUTIVA,NO TIENE,ORAL
